In [2]:
import pandas as pd
import numpy as np
import os
import shutil
from tqdm import tqdm
from glob import glob
import librosa
import warnings
import pickle

warnings.filterwarnings("ignore")

def save_dict_as_txt(d):
    f = open('./Spectrogram/naming.txt', 'w')
    for key, value in d.items():
        f.write(f'{key} : {value} \n')
    f.close()

def find_name():
    with open('./Spectrogram/naming.pickle', 'rb') as fr:
        naming = pickle.load(fr)
    current_list = sorted(list(naming.keys()))
    new_name = current_list[-1] + 1
    return new_name

def add_name(name: int, description: str):
    with open('./Spectrogram/naming.pickle', 'rb') as fr:
        naming = pickle.load(fr)
    assert name not in naming, f"already naming.pickle has {name}!"
    naming[name] = description
    with open('./Spectrogram/naming.pickle','wb') as fw:
        pickle.dump(naming, fw)
    save_dict_as_txt(naming)
        
def delete_name(name: int):
    with open('./Spectrogram/naming.pickle', 'rb') as fr:
        naming = pickle.load(fr)
    assert name in naming, f"naming.pickle has not {name}!"
    del naming[name]
    with open('./Spectrogram/naming.pickle','wb') as fw:
        pickle.dump(naming, fw)
    save_dict_as_txt(naming)

## 0.naive / 1. shuffle 

In [2]:
sample_submission = pd.read_csv("./open/sample_submission.csv")
sample_submission.head()

,id,africa,australia,canada,england,hongkong,us
0,1,0,0,0,0,0,0
1,2,0,0,0,0,0,0
2,3,0,0,0,0,0,0
3,4,0,0,0,0,0,0
4,5,0,0,0,0,0,0


In [5]:
africa_train_data = np.load("./npy_data/africa_npy.npy", allow_pickle = True)
australia_train_data = np.load("./npy_data/australia_npy.npy", allow_pickle = True)
canada_train_data = np.load("./npy_data/canada_npy.npy", allow_pickle = True)
england_train_data = np.load("./npy_data/england_npy.npy", allow_pickle = True)
hongkong_train_data = np.load("./npy_data/hongkong_npy.npy", allow_pickle = True)
us_train_data = np.load("./npy_data/us_npy.npy", allow_pickle = True)

test_data = np.load("./npy_data/test_npy.npy", allow_pickle = True)

train_data_list = [africa_train_data, australia_train_data, canada_train_data, england_train_data, hongkong_train_data, us_train_data]

In [13]:
def get_mini(data):

    mini = 9999999
    for i in data:
        if len(i) < mini:
            mini = len(i)

    return mini


def set_length(data, d_mini):

    result = []
    for i in data:
        result.append(i[:d_mini])
    result = np.array(result)

    return result


def get_feature(data, sr = 16000, n_fft = 256, win_length = 200, hop_length = 160, n_mels = 64):
    mel = []
    for i in tqdm(data):
        mel_ = librosa.feature.melspectrogram(i, sr = sr, n_fft = n_fft, win_length = win_length, hop_length = hop_length, n_mels = n_mels)
        mel.append(mel_)
    mel = np.array(mel)
    mel = librosa.power_to_db(mel, ref = np.max)

    mel_mean = mel.mean()
    mel_std = mel.std()
    mel = (mel - mel_mean) / mel_std

    return mel

In [14]:
train_x = np.concatenate(train_data_list, axis= 0)
test_x = np.array(test_data)

train_mini = get_mini(train_x)
test_mini = get_mini(test_x)

mini = np.min([train_mini, test_mini])

train_x = set_length(train_x, mini)
test_x = set_length(test_x, mini)

train_x = get_feature(data = train_x)
test_x = get_feature(data = test_x)

train_x = train_x.reshape(-1, train_x.shape[1], train_x.shape[2], 1)
test_x = test_x.reshape(-1, test_x.shape[1], test_x.shape[2], 1)

100%|█████████████████████████████████████████████████████████████████████████████| 6100/6100 [00:22<00:00, 268.29it/s]


In [6]:
train_y = np.concatenate((np.zeros(len(africa_train_data), dtype = np.int),
                        np.ones(len(australia_train_data), dtype = np.int),
                         np.ones(len(canada_train_data), dtype = np.int) * 2,
                         np.ones(len(england_train_data), dtype = np.int) * 3,
                         np.ones(len(hongkong_train_data), dtype = np.int) * 4,
                         np.ones(len(us_train_data), dtype = np.int) * 5), axis = 0)
np.save("./data/train_y_npy.npy", train_y)

In [3]:
train_x = np.load("./data/train_x_npy.npy", allow_pickle = True)
train_y = np.load("./data/train_y_npy.npy", allow_pickle = True)
test_x = np.load("./data/test_x_npy.npy", allow_pickle = True)

In [4]:
train_x.shape, train_y.shape, test_x.shape

((25520, 64, 501, 1), (25520,), (6100, 64, 501, 1))

In [5]:
s = np.arange(train_x.shape[0])
np.random.shuffle(s)
s

array([18241,  2954,  9763, ..., 17188, 14450, 22304])

In [6]:
train_x = train_x[s]
train_y = train_y[s]
np.random.shuffle(test_x)

In [7]:
np.save("./data/train_x_shf.npy", train_x)
np.save("./data/train_y_shf.npy", train_y)
np.save("./data/test_x_shf.npy", test_x)

## 2. frequency mask

In [1]:
from audiomentations import SpecFrequencyMask
import numpy as np

In [2]:
train_x_shf = np.load('./data/train_x_shf.npy', allow_pickle=True)
train_x_shf.shape

(25520, 64, 501, 1)

In [3]:
mask = SpecFrequencyMask(p=0.5)

In [8]:
train_x_shf_mask = []
for train_x in train_x_shf:
    train_x_shf_mask.append(mask(train_x))
train_x_shf_mask = np.array(train_x_shf_mask)
train_x_shf_mask.shape

(25520, 64, 501, 1)

In [9]:
np.save("./data/train_x_shf_mask.npy", train_x_shf_mask)

Untill now, I didn't use "naming.pickle" API <br>
After this time, new version of spectrogram will be named by "naming.pickle" API

## 3. add noise

In [3]:
from audiomentations import AddGaussianNoise

In [4]:
def get_mini(data):
    mini = 9999999
    for i in data:
        if len(i) < mini:
            mini = len(i)
    return mini

def set_length(data, d_mini):
    result = []
    for i in data:
        result.append(i[:d_mini])
    result = np.array(result)
    return result

def get_feature(data, sr = 16000, n_fft = 256, win_length = 200, hop_length = 160, n_mels = 64):
    mel = []
    for i in tqdm(data):
        mel_ = librosa.feature.melspectrogram(i, sr = sr, n_fft = n_fft, win_length = win_length, hop_length = hop_length, n_mels = n_mels)
        mel.append(mel_)
    mel = np.array(mel)
    mel = librosa.power_to_db(mel, ref = np.max)

    mel_mean = mel.mean()
    mel_std = mel.std()
    mel = (mel - mel_mean) / mel_std

    return mel

In [5]:
africa_train_data = np.load("./RawData/africa_npy.npy", allow_pickle = True)
australia_train_data = np.load("./RawData/australia_npy.npy", allow_pickle = True)
canada_train_data = np.load("./RawData/canada_npy.npy", allow_pickle = True)
england_train_data = np.load("./RawData/england_npy.npy", allow_pickle = True)
hongkong_train_data = np.load("./RawData/hongkong_npy.npy", allow_pickle = True)
us_train_data = np.load("./RawData/us_npy.npy", allow_pickle = True)

train_data_list = [africa_train_data, australia_train_data, canada_train_data, england_train_data, hongkong_train_data, us_train_data]
train_x = np.concatenate(train_data_list, axis= 0)

test_x = np.load("./RawData/test_npy.npy", allow_pickle = True)

train_mini = get_mini(train_x)
test_mini = get_mini(test_x)

mini = np.min([train_mini, test_mini])

train_x = set_length(train_x, mini)
test_x = set_length(test_x, mini)

In [8]:
train_x.shape, test_x.shape

((25520, 80064), (6100, 80064))

In [12]:
train_x[0].max(), train_x[0].min(), train_x[0].mean(), train_x[0].std()

(0.7335644, -0.6974866, 8.164273e-05, 0.14039339)

In [39]:
# add noise
noise = AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5)
for i, x in enumerate(train_x):
    train_x[i] = noise(x, 16000)

In [40]:
# make spectrogram & train_y
train_x = get_feature(data = train_x)
test_x = get_feature(data = test_x)

train_x = train_x.reshape(-1, train_x.shape[1], train_x.shape[2], 1)
test_x = test_x.reshape(-1, test_x.shape[1], test_x.shape[2], 1)

train_y = np.concatenate((np.zeros(len(africa_train_data), dtype = np.int),
                        np.ones(len(australia_train_data), dtype = np.int),
                         np.ones(len(canada_train_data), dtype = np.int) * 2,
                         np.ones(len(england_train_data), dtype = np.int) * 3,
                         np.ones(len(hongkong_train_data), dtype = np.int) * 4,
                         np.ones(len(us_train_data), dtype = np.int) * 5), axis = 0)

100%|█████████████████████████████████████████████████████████████████████████████| 6100/6100 [00:25<00:00, 237.10it/s]


In [41]:
train_x.shape, test_x.shape, train_y.shape

((25520, 64, 501, 1), (6100, 64, 501, 1), (25520,))

In [42]:
# shuffle
s = np.arange(train_x.shape[0])
np.random.shuffle(s)
train_x = train_x[s]
train_y = train_y[s]

In [43]:
new_name = find_name()
np.save(f"./Spectrogram/train_x_{new_name}.npy", train_x)
np.save(f"./Spectrogram/train_y_{new_name}.npy", train_y)
add_name(new_name, "AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5)")